# Benchmark - Solving -

In [ ]:
import os
import pandas as pd
from time import time
from glob import glob

In [ ]:
import merrin.Loader as loader
import merrin.ASPApplication as app
import merrin.DataProcessing as dataprocessing
from merrin.ASPApplication import ResolutionMode, UpdateMode, OptimisationMode
from merrin.Parameters import FLUXOMIC_DATA, PROTEOMIC_DATA, CINETIC_DATA, DATA_TYPE_COLUMN

## Parameters

In [ ]:
# Files
metabolic_network_file = f'data/covert/metabolic_network.xml'
regulatory_network_file = f'data/covert/regulatory_network.sbml'
pkn_file = f'data/covert/interactions.txt'
objective_function = 'Growth'

In [ ]:
# Model Parameters
threads = 1
limits = 51
timelimits = 600
resolution_mode = ResolutionMode.SUBSET_MINIMAL
update_mode = UpdateMode.SYNCHRONOUS

In [ ]:
# Benchmark
benchmark_dir = './benchmark/instances/datatype-{}/degradation-{}%/seed-{}/*.csv'

# Instance to solve
data_type = [
    [CINETIC_DATA, FLUXOMIC_DATA, PROTEOMIC_DATA],
    [CINETIC_DATA, PROTEOMIC_DATA],
    [PROTEOMIC_DATA],
    [CINETIC_DATA, FLUXOMIC_DATA]
]
seeds = range(0, 10)
degradations = range(0, 60, 10)

# Results
results_dir = './benchmark/results/bnets/'
statistic_file = './benchmark/results/inferring_statistics.csv'
bn_index_file = './benchmark/results/bn_index.csv'
bns_by_instance_file = './benchmark/results/bn_by_instances.csv'

# Parameters
optimisation_mode = OptimisationMode.TIME_BUFFER
observation_buffer = 5
data_error = 0.3

In [ ]:
def create_folder(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

In [ ]:
create_folder('./benchmark/results/')

## Load Files

In [ ]:
mn = loader.load_sbml(metabolic_network_file)
inputs = sorted(mn.get_inputs())
pkn = loader.load_pkn(pkn_file, inputs)

In [ ]:
reactions  = mn.get_reactions()
external_reactions = set(r for r, _ in mn.get_input_reactions())
internal_reactions = reactions.difference(external_reactions)
inputs = mn.get_inputs()
outputs = mn.get_outputs()
regulators = set(pkn.nodes()).difference(reactions).difference(inputs).difference(outputs)

## Solve Benchmark

In [ ]:
def load_data(path):
    sims_file = sorted(glob(path))
    sims = {}
    for k, sf in enumerate(sims_file):
        sims[k] = pd.read_csv(sf, sep=',')
        sims[k].set_index('Time', inplace=True)
    return dataprocessing.preprocess_simulations(sims)
    
def write_bnet(file, bn):
    bnet = []
    for k, v in bn.items():
        if v == '':
            continue
        bnet.append(f'{k}, {v}')
    bnet = '\n'.join(bnet)
    with open(file, 'w') as f:
        f.write(bnet)

In [ ]:
benchmark_data = []
inferred_bn = []
for seed in seeds:
    for degradation in degradations:
        degradation /= 100
        for tags in data_type:
            tags.sort()
            tags_str = ''.join([t[0] for t in tags])
            instances_path = benchmark_dir.format(tags_str, int(degradation * 100), seed)

            print(f'Instance: {tags_str}-{int(degradation * 100)}%-S{seed}')
    
            # Solving
            dt = time()
            processed_observations = load_data(instances_path)
            model = app.Model(
                mn, pkn, objective_function,
                n=limits, thread=threads,
                resolution_mode=resolution_mode, optimisation_mode=optimisation_mode,
                data_error=data_error
            )
            model.build(
                processed_observations, 
                observation_buffer=observation_buffer,
                update_mode=update_mode
            )
            status = model.solve(timelimit=timelimits)
            dt = time() - dt
            
            print(f'\tSolved.')

            results = model.get_results()
            statistics =  model.statistics()

            # Retrieve results
            instance_id = {
                    'Instance': 'covert',
                    'Data type': tags_str.upper(),
                    'Degradation %': int(degradation * 100),
                    'Seed': seed,
            }
            benchmark_data.append(
                instance_id |
                {
                    'Error epsilon': data_error,
                    'Observation buffer': observation_buffer,
                    'Time limits': timelimits,
                    'Solution limit (s)': limits,
                    'Status': status,
                    '# Results': len(results),
                    'Time (s)': dt,
                    "#Call to MSS_verification": statistics['MSS_verification']['nb_calls'],
                    "Total time in MSS_verification (s)": statistics['MSS_verification']['total_duration'],
                    "#Call to REG_verification": statistics['REG_verification']['nb_calls'],
                    "Total time in REG_verification (s)": statistics['REG_verification']['total_duration']
                }
            )

            # Save solutions
            for i, res in enumerate(results):
                extended_res = instance_id | {'Solution ID': i} | res
                inferred_bn.append(extended_res)
                # bnet_file = export_path + f'solution-{i}.bnet'
                # write_bnet(bnet_file, res)

            print(f'\tDone.')

## Exporting statistics

In [ ]:
df = pd.DataFrame(benchmark_data)
df = df.sort_values(['Instance', 'Data type', 'Degradation %', 'Seed'])
df = df.reset_index()
df = df.drop('index', axis=1)
df = df.rename({'Data type': 'Data_type', 'Degradation %': 'Degradation', 'Solution ID': 'Solution_ID'}, axis=1)
df.to_csv(statistic_file, header=True, index=False)
df

## Indexing the inferred Boolean networks

### CSV export

In [ ]:
bn_index = pd.DataFrame(inferred_bn)[list(pkn.nodes)]
bn_index = bn_index.drop_duplicates()
bn_index = bn_index.reset_index()
bn_index = bn_index.drop('index', axis=1)
bn_index = bn_index.reset_index()
bn_index

In [ ]:
bns_by_instance = pd.DataFrame(inferred_bn)
bns_by_instance = bns_by_instance.merge(bn_index, on=list(pkn.nodes), how='left')
bns_by_instance = bns_by_instance.rename(columns={'index': 'Ref_BN'})
bns_by_instance = bns_by_instance.rename({'Data type': 'Data_type', 'Degradation %': 'Degradation', 'Solution ID': 'Solution_ID'}, axis=1)
bns_by_instance  = bns_by_instance.drop(list(pkn.nodes), axis=1)
bns_by_instance

In [ ]:
bn_index.to_csv(bn_index_file)
bns_by_instance.to_csv(bns_by_instance_file)

### BNet generation

In [ ]:
for bn_data in bn_index[list(pkn.nodes)].iterrows():
    ref = bn_data[0]
    bn = bn_data[1]
    bnet_path = f'./benchmark/results/bnet/'
    create_folder(bnet_path)
    write_bnet(bnet_path + f'BN-{ref}.bnet', bn)